## Deliverable 3. Create a Travel Itinerary Map.

In [98]:
# Dependencies and Setup
import pandas as pd
import os
import requests
import gmaps

# Import API key
from config2 import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [97]:
os.chdir('Unit 2 - Python/World_Weather_Analysis')


In [99]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("Vacation_Search/HotelCities.csv")
vacation_df.head()


,City_ID,Hotel Name,City,Country,Max Temp,Lat,Lng
0,2,Tu Casa,East London,ZA,26.74,-33.02,27.91
1,3,People ThankYou,Rikitea,PF,26.07,-23.12,-134.97
2,4,Villa Enata,Atuona,PF,25.09,-9.80,-139.03
3,5,Sheraton Kauai Coconut Beach Resort,Kapaa,US,19.99,22.08,-159.32
4,8,Hotel Posada San Jose,Pochutla,MX,28.90,15.74,-96.47


In [100]:
weather_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
weather_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Sistranda,NO,63.73,8.83,275.32,85,100,6.12,moderate rain
1,1,Mataura,NZ,-46.19,168.86,283.97,91,100,1.41,overcast clouds
2,2,East London,ZA,-33.02,27.91,299.89,80,0,7.30,clear sky
3,3,Rikitea,PF,-23.12,-134.97,299.22,71,48,7.10,light rain
4,4,Atuona,PF,-9.80,-139.03,298.24,79,10,6.12,light rain


In [101]:
# I thought I needed hotel names!  Oh well, problem solved.
vacation_df["Weather Description"] = weather_df["Weather Description"]

In [102]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °C</dd>
<dt>Weather</dt><dd>{Weather Description}</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]

In [103]:
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# 4a. Add a marker layer for each city to the map.
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 4b. Display the figure
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))

In [104]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

vacation_start = vacation_df.loc[vacation_df["City"] == "Abancay"]
vacation_end = vacation_start
vacation_stop1 = vacation_df.loc[vacation_df["City"] == "San Ramon De La Nueva Oran"]
vacation_stop2 = vacation_df.loc[vacation_df["City"] == "Vilhena"] 
vacation_stop3 = vacation_df.loc[vacation_df["City"] == "Sao Felix Do Xingu"] 

In [105]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.

start = vacation_start[['Lat','Lng']].to_numpy(dtype ='float32')
end = start
stop1 = vacation_stop1[['Lat','Lng']].to_numpy(dtype ='float32')
stop2 = vacation_stop2[['Lat','Lng']].to_numpy(dtype ='float32')
stop3 = vacation_stop3[['Lat','Lng']].to_numpy(dtype ='float32')
end[0]

array([-13.63, -72.88], dtype=float32)

In [106]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
fig = gmaps.figure()
route = gmaps.directions_layer(start[0],end[0],waypoints=[stop1[0],stop2[0],stop3[0]],travel_mode='DRIVING')
fig.add_layer(route)
# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))

In [107]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, vacation_stop3],ignore_index=True)
itinerary_df

,City_ID,Hotel Name,City,Country,Max Temp,Lat,Lng,Weather Description
0,653,Siball Hotel,Abancay,PE,16.58,-13.63,-72.88,thunderstorm
1,207,Hotel Residencial Centro,San Ramon De La Nueva Oran,AR,18.32,-23.13,-64.33,overcast clouds
2,405,Hotel Mirage,Vilhena,BR,27.12,-12.74,-60.15,overcast clouds
3,700,Rio Xingu Camping,Sao Felix Do Xingu,BR,22.19,-6.64,-52.00,light snow


In [108]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} K</dd>
<dt>Weather</dt><dd>{Weather Description}</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [114]:
# 11a. Add a marker layer for each city to the map.
marker_layer = gmaps.marker_layer(locations, info_box_template)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
Weather Description